In [12]:
!pip uninstall tensorflow
!pip install tensorflow==2.13.0

Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.15.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-2.15.0
  Using cached tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (524.1 MB)
  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached tensorflow_estimator-2.13.0-py2.py3-none-any.whl (440 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimat

In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

# Display the version
print(tf.__version__)

# import and mount Drive
import os
from google.colab import drive
drive.mount('/content/gdrive')

2.13.0
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# get path to folder file is in
file_path = '/content/gdrive/MyDrive/Orthogonal'
print(file_path)

/content/gdrive/MyDrive/Orthogonal


In [7]:
# Download the data from tf, unless it's already here.
if not os.path.exists(file_path+'/data/clean'):
  os.mkdir(file_path+'/data/clean')
  (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
  np.save(file_path+'/data/clean/X_train.npy',X_train)
  np.save(file_path+'/data/clean/y_train.npy',y_train)
  np.save(file_path+'/data/clean/X_test.npy',X_test)
  np.save(file_path+'/data/clean/y_test.npy',y_test)
else:
  X_train = np.load(file_path+'/data/clean/X_train.npy')
  y_train = np.load(file_path+'/data/clean/y_train.npy')
  X_test = np.load(file_path+'/data/clean/X_test.npy')
  y_test = np.load(file_path+'/data/clean/y_test.npy')

print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)
print("X_test shape", X_test.shape)
print("y_test shape", y_test.shape)

X_train shape (50000, 32, 32, 3)
y_train shape (50000, 1)
X_test shape (10000, 32, 32, 3)
y_test shape (10000, 1)


In [8]:
# Convert to float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Decrease pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

In [9]:
# Create FGSM Perturbed Data
def create_fgsm_perturbations(data_inputs, data_labels, epsilon = 0.1):
  input_tensor = tf.convert_to_tensor(data_inputs)
  label_tensor = tf.convert_to_tensor(data_labels)
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
  with tf.GradientTape() as tape:
    tape.watch(input_tensor)
    output_tensor = model(input_tensor)
    loss = loss_fn(label_tensor, output_tensor)

  gradient = tape.gradient(loss, input_tensor)
  perturbed_input_tensor = input_tensor + epsilon * tf.sign(gradient)
  return tf.cast(perturbed_input_tensor, dtype=input_tensor.dtype)

# Create FGSM perturbations if nonexisting, else load them
if not os.path.exists(file_path+'/data/FGSM_perturbed'):
  os.mkdir(file_path+'/data/FGSM_perturbed')

  perturbed_data = [None for _ in range(5)]
  for i in range(5):
    perturbed_data[i] = create_fgsm_perturbations(X_train[i * 10000:(i+1) * 10000], y_train[i * 10000:(i+1) * 10000])

  X_FGSM_train = np.concatenate(perturbed_data, axis=0)
  X_FGSM_test = create_fgsm_perturbations(X_test, y_test)

  np.save(file_path+'/data/FGSM_perturbed/X_FGSM_train.npy',X_FGSM_train)
  np.save(file_path+'/data/FGSM_perturbed/X_FGSM_test.npy',X_FGSM_test)
else:
  X_FGSM_train = np.load(file_path+'/data/FGSM_perturbed/X_FGSM_train.npy')
  X_FGSM_test = np.load(file_path+'/data/FGSM_perturbed/X_FGSM_test.npy')

print("X_FGSM_train shape", X_FGSM_train.shape)
print("y_train shape", y_train.shape)
print("X_FGSM_test shape", X_FGSM_test.shape)
print("y_test shape", y_test.shape)

X_FGSM_train shape (50000, 32, 32, 3)
y_train shape (50000, 1)
X_FGSM_test shape (10000, 32, 32, 3)
y_test shape (10000, 1)


In [10]:
# Clip perturbed training and test data to viable values
X_FGSM_train = np.clip(X_FGSM_train, a_min = 0.0, a_max = 1.0)
X_FGSM_test = np.clip(X_FGSM_test, a_min = 0.0, a_max = 1.0)

In [11]:
# Create PGD Perturbed Data
def create_pgd_perturbations(data_inputs, data_labels, epsilon = 0.01, n_iter = 10):
  input_tensor = tf.convert_to_tensor(data_inputs)
  label_tensor = tf.convert_to_tensor(data_labels)
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)

  perturbed_tensor = input_tensor
  for i in range(n_iter):
      with tf.GradientTape() as tape:
          tape.watch(perturbed_tensor)
          output_tensor = model(perturbed_tensor)
          loss = loss_fn(label_tensor, output_tensor)

      gradient = tape.gradient(loss, perturbed_tensor)

      perturbed_tensor = perturbed_tensor + gradient
      # Project perturbed_tensor onto the L-infinity ball around input_tensor
      perturbed_tensor = epsilon * tf.sign(
          perturbed_tensor - input_tensor) + input_tensor

  perturbed_tensor = tf.cast(perturbed_tensor, dtype=input_tensor.dtype)
  return perturbed_tensor

# Create PGD perturbations if nonexisting, else load them
if not os.path.exists(file_path+'/data/PGD_perturbed'):
  os.mkdir(file_path+'/data/PGD_perturbed')

  perturbed_data = [None for _ in range(5)]
  for i in range(5):
    perturbed_data[i] = create_fgsm_perturbations(X_train[i * 10000:(i+1) * 10000], y_train[i * 10000:(i+1) * 10000])

  X_PGD_train = np.concatenate(perturbed_data, axis=0)
  X_PGD_test = create_pgd_perturbations(X_test, y_test)

  np.save(file_path+'/data/PGD_perturbed/X_PGD_train.npy',X_PGD_train)
  np.save(file_path+'/data/PGD_perturbed/X_PGD_test.npy',X_PGD_test)
else:
  X_PGD_train = np.load(file_path+'/data/PGD_perturbed/X_PGD_train.npy')
  X_PGD_test = np.load(file_path+'/data/PGD_perturbed/X_PGD_test.npy')

print("X_PGD_train shape", X_PGD_train.shape)
print("y_train shape", y_train.shape)
print("X_PGD_test shape", X_PGD_test.shape)
print("y_test shape", y_test.shape)

X_PGD_train shape (50000, 32, 32, 3)
y_train shape (50000, 1)
X_PGD_test shape (10000, 32, 32, 3)
y_test shape (10000, 1)


In [12]:
# Clip perturbed training and test data to viable values
X_PGD_train = np.clip(X_PGD_train, a_min = 0.0, a_max = 1.0)
X_PGD_test = np.clip(X_PGD_test, a_min = 0.0, a_max = 1.0)

In [4]:
def evaluate(model):
  # Evaluate on Clean Test Data
  print("clean test data: ")
  model.evaluate(X_test, y_test)

  # Evaluate on FGSM-perturbed test data
  print("fgsm perturbed test data: ")
  model.evaluate(X_FGSM_test, y_test)

  # Evaluate on PGD-perturbed test data
  print("pgd perturbed test data: ")
  model.evaluate(X_PGD_test, y_test)

def evaluateDenoised(model, autoencoder):
  # Evaluate on (Denoised) Clean Test Data
  print("clean test data: ")
  model.evaluate(autoencoder.predict(X_test), y_test)

  # Evaluate on (Denoised) FGSM-perturbed test data
  print("fgsm perturbed test data: ")
  model.evaluate(autoencoder.predict(X_FGSM_test), y_test)

  # Evaluate on (Denoised) PGD-perturbed test data
  print("pgd perturbed test data: ")
  model.evaluate(autoencoder.predict(X_PGD_test), y_test)

In [10]:
def evaluateBothDenoised(model, autoencoder1, autoencoder2):
  denoised_X_test = autoencoder2.predict(autoencoder1.predict(X_test))
  # Evaluate on (Denoised) Clean Test Data
  print("clean test data: ")
  model.evaluate(denoised_X_test, y_test)

  denoised_X_test = autoencoder2.predict(autoencoder1.predict(X_FGSM_test))
  # Evaluate on (Denoised) FGSM-perturbed test data
  print("fgsm perturbed test data: ")
  model.evaluate(denoised_X_test, y_test)

  denoised_X_test = autoencoder2.predict(autoencoder1.predict(X_PGD_test))
  # Evaluate on (Denoised) PGD-perturbed test data
  print("pgd perturbed test data: ")
  model.evaluate(denoised_X_test, y_test)

In [11]:
# Save Denoised Data
def saveDenoisedData(autoencoder, typeDenoising):
  if not os.path.exists(file_path+'/data/denoised'):
    os.mkdir(file_path+'/data/denoised')
  np.save(file_path+'/data/denoised/'+typeDenoising+'_denoised_X_test.npy',autoencoder.predict(X_test))
  np.save(file_path+'/data/denoised/'+typeDenoising+'_denoised_X_FGSM_test.npy',autoencoder.predict(X_FGSM_test))
  np.save(file_path+'/data/denoised/'+typeDenoising+'_denoised_X_PGD_test.npy',autoencoder.predict(X_PGD_test))


In [12]:
# Save FGSM Denoised Data
FGSM_denoising_autoencoder = tf.keras.models.load_model(file_path+'/models/FGSM_denoising_autoencoder2.keras')
saveDenoisedData(FGSM_denoising_autoencoder, 'FGSM')

# Save PGD Denoised Data
PGD_denoising_autoencoder = tf.keras.models.load_model(file_path+'/models/PGD_denoising_autoencoder2.keras')
saveDenoisedData(PGD_denoising_autoencoder, 'PGD')

313/313 [==============================] - 17s 55ms/step


In [12]:
# Load in Clean Model
print("clean model:\n")
model = tf.keras.models.load_model(file_path+'/models/clean.keras')
evaluate(model)

# Load in FGSM Model
print("\nfgsm model:\n")
fgsm_model = tf.keras.models.load_model(file_path+'/models/mixed_FGSM.keras')
evaluate(fgsm_model)

# Load in PGD Model
print("\npgd model:\n")
pgd_model = tf.keras.models.load_model(file_path+'/models/mixed_PGD.keras')
evaluate(pgd_model)

# Load in FGSM Denoising Autoencoder
print("\ndenoised fgsm noise, clean model:\n")
FGSM_denoising_autoencoder = tf.keras.models.load_model(file_path+'/models/FGSM_denoising_autoencoder2.keras')
evaluateDenoised(model, FGSM_denoising_autoencoder)

# Load in PGD Denoising Autoencoder
print("\ndenoised pgd noise, clean model:\n")
PGD_denoising_autoencoder = tf.keras.models.load_model(file_path+'/models/PGD_denoising_autoencoder2.keras')
evaluateDenoised(model, PGD_denoising_autoencoder)

def temperature_cross_entropy(gt, pred):
  loss = tf.nn.softmax_cross_entropy_with_logits(labels=gt, logits=pred/20)
  return loss

# Load in Distilled Model
print("\ndistilled model:\n")
distilled_model = tf.keras.models.load_model(file_path+'/models/distillation_student.keras', custom_objects={'temperature_cross_entropy': temperature_cross_entropy})
evaluate(distilled_model)

clean model:

clean test data: 
313/313 [==============================] - 8s 23ms/step - loss: 0.7490 - accuracy: 0.8142
fgsm perturbed test data: 
313/313 [==============================] - 8s 27ms/step - loss: 6.7091 - accuracy: 0.0394
pgd perturbed test data: 
313/313 [==============================] - 8s 25ms/step - loss: 2.9631 - accuracy: 0.3434

fgsm model:

clean test data: 
313/313 [==============================] - 11s 34ms/step - loss: 0.9616 - accuracy: 0.7367
fgsm perturbed test data: 
313/313 [==============================] - 9s 29ms/step - loss: 0.7698 - accuracy: 0.8128
pgd perturbed test data: 
313/313 [==============================] - 9s 30ms/step - loss: 1.0124 - accuracy: 0.7206

pgd model:

clean test data: 
313/313 [==============================] - 8s 25ms/step - loss: 0.9464 - accuracy: 0.7493
fgsm perturbed test data: 
313/313 [==============================] - 11s 34ms/step - loss: 0.7472 - accuracy: 0.8175
pgd perturbed test data: 
313/313 [===============

In [16]:
# (FGSM + PGD) Adversarially trained classifier
print("(FGSM + PGD) Adversarially Trained model:\n")
mixed_all_model = tf.keras.models.load_model(file_path+'/models/mixed_all.keras')
evaluate(mixed_all_model)

# (FGSM + PGD) Adversarial Noise Reduction
print("\n(FGSM + PGD) Adversarial Noise Reduction models\n")
evaluateBothDenoised(model, FGSM_denoising_autoencoder, PGD_denoising_autoencoder)

# Defensive Distillation + Random Self Ensemble
print("\nDefensive Distillation + Random Self Ensemble model\n")
distilled_rse = tf.keras.models.load_model(file_path+'/models/distilled_rse.keras', custom_objects={'temperature_cross_entropy': temperature_cross_entropy})
evaluate(distilled_rse)

(FGSM + PGD) Adversarially Trained model:

clean test data: 
313/313 [==============================] - 9s 27ms/step - loss: 0.9812 - accuracy: 0.7221
fgsm perturbed test data: 
313/313 [==============================] - 11s 35ms/step - loss: 0.5467 - accuracy: 0.8806
pgd perturbed test data: 
313/313 [==============================] - 11s 35ms/step - loss: 1.0106 - accuracy: 0.7158

(FGSM + PGD) Adversarial Noise Reduction models

313/313 [==============================] - 18s 56ms/step
clean test data: 
313/313 [==============================] - 16s 51ms/step
fgsm perturbed test data: 
313/313 [==============================] - 16s 52ms/step
pgd perturbed test data: 
313/313 [==============================] - 11s 35ms/step - loss: 1.9515 - accuracy: 0.4983

Defensive Distillation + Random Self Ensemble model

clean test data: 
313/313 [==============================] - 9s 26ms/step - loss: 276.0089 - accuracy: 0.7336
fgsm perturbed test data: 
313/313 [==============================]

In [18]:
# (FGSM) Adversarially Trained Classifier + (FGSM) Adversarial Noise Reduction
print("\n(FGSM) Adversarially Trained Classifier + (FGSM) Adversarial Noise Reduction models\n")
evaluateDenoised(fgsm_model, FGSM_denoising_autoencoder)

# (FGSM) Adversarially Trained Classifier + (PGD) Adversarial Noise Reduction
print("\n(FGSM) Adversarially Trained Classifier + (PGD) Adversarial Noise Reduction models\n")
evaluateDenoised(fgsm_model, PGD_denoising_autoencoder)

# (FGSM) Adversarially Trained Classifier + Defensive Distillation
print("\n(FGSM) Adversarially Trained Classifier + Defensive Distillation model\n")
fgsm_distilled_model = tf.keras.models.load_model(file_path+'/models/distilled_FGSM_mixed.keras', custom_objects={'temperature_cross_entropy': temperature_cross_entropy})
evaluate(fgsm_distilled_model)


(FGSM) Adversarially Trained Classifier + (FGSM) Adversarial Noise Reduction models

clean test data: 
313/313 [==============================] - 8s 26ms/step - loss: 1.3107 - accuracy: 0.6272
fgsm perturbed test data: 
313/313 [==============================] - 9s 28ms/step - loss: 1.8894 - accuracy: 0.4604
pgd perturbed test data: 
313/313 [==============================] - 9s 29ms/step - loss: 1.3532 - accuracy: 0.6159

(FGSM) Adversarially Trained Classifier + (PGD) Adversarial Noise Reduction models

clean test data: 
313/313 [==============================] - 9s 29ms/step - loss: 1.2836 - accuracy: 0.6329
fgsm perturbed test data: 
313/313 [==============================] - 8s 24ms/step - loss: 1.8579 - accuracy: 0.4734
pgd perturbed test data: 
313/313 [==============================] - 10s 32ms/step - loss: 1.3268 - accuracy: 0.6200

(FGSM) Adversarially Trained Classifier + Defensive Distillation model

clean test data: 
313/313 [==============================] - 7s 22ms/step

In [19]:
# (PGD) Adversarially Trained Classifier + (FGSM) Adversarial Noise Reduction
print("\n(PGD) Adversarially Trained Classifier + (FGSM) Adversarial Noise Reduction models\n")
evaluateDenoised(pgd_model, FGSM_denoising_autoencoder)

# (PGD) Adversarially Trained Classifier + (PGD) Adversarial Noise Reduction
print("\n(PGD) Adversarially Trained Classifier + (PGD) Adversarial Noise Reduction models\n")
evaluateDenoised(pgd_model, PGD_denoising_autoencoder)

# (PGD) Adversarially Trained Classifier + Defensive Distillation
print("\n(PGD) Adversarially Trained Classifier + Defensive Distillation model\n")
pgd_distilled_model = tf.keras.models.load_model(file_path+'/models/distilled_PGD_mixed.keras', custom_objects={'temperature_cross_entropy': temperature_cross_entropy})
evaluate(pgd_distilled_model)


(PGD) Adversarially Trained Classifier + (FGSM) Adversarial Noise Reduction models

clean test data: 
313/313 [==============================] - 8s 26ms/step - loss: 1.2695 - accuracy: 0.6410
fgsm perturbed test data: 
313/313 [==============================] - 8s 27ms/step - loss: 1.8853 - accuracy: 0.4688
pgd perturbed test data: 
313/313 [==============================] - 9s 28ms/step - loss: 1.3159 - accuracy: 0.6241

(PGD) Adversarially Trained Classifier + (PGD) Adversarial Noise Reduction models

clean test data: 
313/313 [==============================] - 7s 24ms/step - loss: 1.2550 - accuracy: 0.6483
fgsm perturbed test data: 
313/313 [==============================] - 7s 21ms/step - loss: 1.8814 - accuracy: 0.4771
pgd perturbed test data: 
313/313 [==============================] - 7s 23ms/step - loss: 1.3018 - accuracy: 0.6317

(PGD) Adversarially Trained Classifier + Defensive Distillation model

clean test data: 
313/313 [==============================] - 7s 21ms/step - l

In [21]:
# (FGSM) Adversarial Noise Reduction + Defensive Distillation
print("\n(FGSM) Adversarial Noise Reduction + Defensive Distillation\n")
evaluateDenoised(distilled_model, FGSM_denoising_autoencoder)

# (PGD) Adversarial Noise Reduction + Defensive Distillation
print("\n(PGD) Adversarial Noise Reduction + Defensive Distillation\n")
evaluateDenoised(distilled_model, PGD_denoising_autoencoder)


(FGSM) Adversarial Noise Reduction + Defensive Distillation

clean test data: 
313/313 [==============================] - 8s 26ms/step - loss: 206.7929 - accuracy: 0.5625
fgsm perturbed test data: 
313/313 [==============================] - 8s 27ms/step - loss: 192.1577 - accuracy: 0.3159
pgd perturbed test data: 
313/313 [==============================] - 7s 23ms/step - loss: 204.9275 - accuracy: 0.5398

(PGD) Adversarial Noise Reduction + Defensive Distillation

clean test data: 
313/313 [==============================] - 7s 22ms/step - loss: 205.8955 - accuracy: 0.5861
fgsm perturbed test data: 
313/313 [==============================] - 9s 27ms/step - loss: 189.5735 - accuracy: 0.3178
pgd perturbed test data: 
313/313 [==============================] - 9s 28ms/step - loss: 203.7760 - accuracy: 0.5598


In [13]:
!pip uninstall tensorflow
!pip install tensorflow==2.15.0

Found existing installation: tensorflow 2.13.0
Uninstalling tensorflow-2.13.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.13.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-2.13.0
  Using cached tensorflow-2.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (475.2 MB)
  Using cached tensorboard-2.15.2-py3-none-any.whl (5.5 MB)
  Using cached tensorflow_estimator-2.15.0-py2.py3-none-any.whl (441 kB)
  Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimat

In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

# Display the version
print(tf.__version__)

# import and mount Drive
import os
from google.colab import drive
drive.mount('/content/gdrive')

2.15.0
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# get path to folder file is in
file_path = '/content/gdrive/MyDrive/Orthogonal'
print(file_path)

/content/gdrive/MyDrive/Orthogonal


In [13]:
# Load in Random Self Ensembled Model
print("\nRSE model:\n")
rse_model = tf.keras.models.load_model(file_path+'/models/rse.keras')
evaluate(rse_model)

# (FGSM) Adversarially Trained Classifier + Random Self Ensemble
print("\n(FGSM) Adversarially Trained Classifier + Random Self Ensemble model\n")
fgsm_rse_model = tf.keras.models.load_model(file_path+'/models/rse_FGSM_perturbed.keras')
evaluate(fgsm_rse_model)

# (PGD) Adversarially Trained Classifier + Random Self Ensemble
print("\n(PGD) Adversarially Trained Classifier + Random Self Ensemble model\n")
pgd_rse_model = tf.keras.models.load_model(file_path+'/models/rse_PGD_perturbed.keras')
evaluate(pgd_rse_model)


RSE model:

clean test data: 
313/313 [==============================] - 3s 4ms/step - loss: 0.9544 - accuracy: 0.7299
fgsm perturbed test data: 
313/313 [==============================] - 1s 4ms/step - loss: 2.7194 - accuracy: 0.3091
pgd perturbed test data: 
313/313 [==============================] - 3s 8ms/step - loss: 1.0608 - accuracy: 0.6951

(FGSM) Adversarially Trained Classifier + Random Self Ensemble model

clean test data: 
313/313 [==============================] - 2s 4ms/step - loss: 1.1565 - accuracy: 0.6665
fgsm perturbed test data: 
313/313 [==============================] - 2s 5ms/step - loss: 1.2086 - accuracy: 0.6588
pgd perturbed test data: 
313/313 [==============================] - 1s 3ms/step - loss: 1.1607 - accuracy: 0.6653

(PGD) Adversarially Trained Classifier + Random Self Ensemble model

clean test data: 
313/313 [==============================] - 1s 3ms/step - loss: 1.1426 - accuracy: 0.6831
fgsm perturbed test data: 
313/313 [===========================

In [14]:
def evaluateDenoisedWData(model, typeDenoising):
  denoised_X_test = np.load(file_path+'/data/denoised/'+typeDenoising+'_denoised_X_test.npy')
  # Evaluate on (Denoised) Clean Test Data
  print("clean test data: ")
  model.evaluate(denoised_X_test, y_test)

  denoised_X_test = np.load(file_path+'/data/denoised/'+typeDenoising+'_denoised_X_FGSM_test.npy')
  # Evaluate on (Denoised) FGSM-perturbed test data
  print("fgsm perturbed test data: ")
  model.evaluate(denoised_X_test, y_test)

  denoised_X_test = np.load(file_path+'/data/denoised/'+typeDenoising+'_denoised_X_PGD_test.npy')
  # Evaluate on (Denoised) PGD-perturbed test data
  print("pgd perturbed test data: ")
  model.evaluate(denoised_X_test, y_test)

In [15]:
# (FGSM) Adversarial Noise Reduction + Random Self Ensemble
print("\n(FGSM) Adversarial Noise Reduction + Random Self Ensemble\n")
evaluateDenoisedWData(rse_model, 'FGSM')

# (PGD) Adversarial Noise Reduction + Random Self Ensemble
print("\n(PGD) Adversarial Noise Reduction + Random Self Ensemble\n")
evaluateDenoisedWData(rse_model, 'PGD')


(FGSM) Adversarial Noise Reduction + Random Self Ensemble

clean test data: 
313/313 [==============================] - 1s 4ms/step - loss: 1.3532 - accuracy: 0.6002
fgsm perturbed test data: 
313/313 [==============================] - 1s 3ms/step - loss: 1.9041 - accuracy: 0.4383
pgd perturbed test data: 
313/313 [==============================] - 1s 3ms/step - loss: 1.3989 - accuracy: 0.5855

(PGD) Adversarial Noise Reduction + Random Self Ensemble

clean test data: 
313/313 [==============================] - 1s 3ms/step - loss: 1.3213 - accuracy: 0.6045
fgsm perturbed test data: 
313/313 [==============================] - 1s 5ms/step - loss: 1.8837 - accuracy: 0.4395
pgd perturbed test data: 
313/313 [==============================] - 1s 3ms/step - loss: 1.3673 - accuracy: 0.5888
